Rozkład LU

Funkcje pomocnicze (suma po k od 1 do i-1 elementów Ai,k * Bk,j oraz suma po k od 1 do i-1 elementów Aj,k * Bk,i)

Rozkład LU macierzy A

In [1]:
function LU_decomposition(A)
    n = size(A)[1]
    L = zeros(Float64,n,n)
    U = zeros(Float64,n,n)
    function sum_of_mults(A,B,i,j)
        s = Float64(0)
        for k = 1:i-1
            s += A[i,k] * B[k,j]
        end
        return s
    end
    function sum_of_mults2(A,B,i,j)
        s = Float64(0)
        for k = 1:i-1
            s += A[j,k] * B[k,i]
        end
        return s
    end
    for i = 1:n
        U[i,i] = A[i,i] - sum_of_mults(L,U,i,i)
        L[i,i] = 1
        for j = (i+1):n
            U[i,j] = A[i,j] - sum_of_mults(L,U,i,j)
            L[j,i] = 1/U[i,i] * (A[j,i] - sum_of_mults2(L,U,i,j))
        end
    end
    return L,U
end
    

LU_decomposition (generic function with 1 method)

Przykład działania

In [2]:
A = [[1 4 7];[2 5 8];[3 6 9]]

3×3 Array{Int64,2}:
 1  4  7
 2  5  8
 3  6  9

In [3]:
L,U = LU_decomposition(A)

([1.0 0.0 0.0; 2.0 1.0 0.0; 3.0 2.0 1.0], [1.0 4.0 7.0; 0.0 -3.0 -6.0; 0.0 0.0 0.0])

In [4]:
L

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 2.0  1.0  0.0
 3.0  2.0  1.0

In [5]:
U

3×3 Array{Float64,2}:
 1.0   4.0   7.0
 0.0  -3.0  -6.0
 0.0   0.0   0.0

In [6]:
L*U

3×3 Array{Float64,2}:
 1.0  4.0  7.0
 2.0  5.0  8.0
 3.0  6.0  9.0

No działa

Funkcja do rozwiązywania układów równań A \* X = B za pomocą metory LU L\*(U\*X) = B

In [7]:
function solve(L,U,B)
    n = size(A)[1]
    Y = zeros(Float64,n,1)
    X = zeros(Float64,n,1)
    Y[1] = B[1]
    function sum(L,Y,i)
        s = Float64(0)
        for j = 1:i-1
            s += L[i,j] * Y[j]
        end
        return s
    end
    for i = 2:n
        Y[i] = B[i] - sum(L,Y,i)
    end
    function sum2(U,X,i,n)
        s = Float64(0)
        for j = (i+1):n
            s += U[i,j] * X[j]
        end
        return s
    end
    X[n] = Y[n] / U[n,n]
    for i = (n-1):-1:1
        X[i] = 1/U[i,i] * (Y[i] - sum2(U,X,i,n))
    end
    return X
end 

solve (generic function with 1 method)

Test

In [8]:
A2 = [[4 -2 4 -2];[3 1 4 2];[2 4 2 1];[2 -2 4 2]]

4×4 Array{Int64,2}:
 4  -2  4  -2
 3   1  4   2
 2   4  2   1
 2  -2  4   2

In [9]:
B = [8; 7; 10; 2]

4-element Array{Int64,1}:
  8
  7
 10
  2

In [10]:
L2,U2 = LU_decomposition(A2)

([1.0 0.0 0.0 0.0; 0.75 1.0 0.0 0.0; 0.5 2.0 1.0 0.0; 0.5 -0.4 -1.2 1.0], [4.0 -2.0 4.0 -2.0; 0.0 2.5 1.0 3.5; 0.0 0.0 -2.0 -5.0; 0.0 0.0 0.0 -1.6])

In [11]:
solve(L2,U2,B)

3×1 Array{Float64,2}:
  4.6               
  1.2000000000000002
 -2.0               

Też działa

Wyznaczanie A^-1 za pomocą rozkładu LU

Niech X = A^-1

In [12]:
function invertible(A)
    n = size(A)[1]
    L,U = LU_decomposition(A)
    B = zeros(Float64,n)
    B[1] = 1
    X = solve(L,U,B)
    for i = 2:n
        B = zeros(Float64,n)
        B[i] = 1
        X = [X (solve(L,U,B))]
    end
    return X
end
    

invertible (generic function with 1 method)

In [270]:
A = [[2 -2 18];[2 1 0];[1 2 0]]

3×3 Array{Int64,2}:
 2  -2  18
 2   1   0
 1   2   0

In [53]:
using LinearAlgebra

In [54]:
function _norm(v)
    n = 0
    for i = 1:length(v)
        n += v[i]*v[i]
    end
    return sqrt(n)
end
#norm([1,2,3]) #powinno 14

_norm (generic function with 1 method)

To też działa :O

In [380]:
function householder(A, k) #k jest którą macierz Householdera liczymy
    n = size(A)[1]
    v = A[k:n,k]
    v[1] = (sign(A[k,k])*norm(v)) + v[1]
    vt = transpose(v)
    H = -2 * v*vt / (vt*v) + I
    ret = zeros(n,n)
    for i = 1:k-1
        ret[i,i] = 1
    end
    for i = k:n
        for j = k:n
            ret[i,j] = H[i-k+1,j-k+1]
        end
    end
    return ret
end

#############################TEST
B =  [1 2 3.0;
      4 5 6;
      7 8 9]
#  H1 = householder(B,1)
#  B = H1*B

# H1 = householder(A, 1)
# A = H1*A
# H2 = householder(A, 2)
# A = H2*A
#H3 = householder(H2, 3)

3×3 Array{Float64,2}:
 1.0  2.0  3.0
 4.0  5.0  6.0
 7.0  8.0  9.0

In [381]:
function QRfactorize(A) #rozkład na macierze Q,R takie, że QR=A
    n = size(A)[1]
    Q = I
    H = A
    for i = 1:n
        H = householder(A, i)
        A = H * A
        Q = Q*H
    end
    return Q,A
end   
############################TEST
B =  [1 2 3.0;
      4 5 6;
      7 8 9]
_B = QRfactorize(B)
_B[1] * _B[2]


3×3 Array{Float64,2}:
 1.0  2.0  3.0
 4.0  5.0  6.0
 7.0  8.0  9.0

In [395]:
function inv_tri(B) #odwrotność macierzy górnotrójkątnej B w O(n^3)
    n = size(B)[1]
    A = deepcopy(B)
    for i = 1:n
        A[i,i] = 1/A[i,i]
    end
    for i = n-1:-1:1
        for j = n:-1:i+1
            s = 0.0
            for k = i+1:j
                s += A[i,k]*A[k,j]
            end
            A[i,j] = - A[i,i]*s
        end
    end
    return A
end
########################TEST
B =  [1 2 3;
      0 5 6;
      0 0 9.0]
inv_tri(inv_tri(B))

#  1.0  -0.4  -0.0666667
#  0.0   0.2  -0.133333 
#  0.0   0.0   0.111111

3×3 Array{Float64,2}:
 1.0  2.0  3.0
 0.0  5.0  6.0
 0.0  0.0  9.0

In [416]:
function QRinv(A)
    Q,R = QRfactorize(A)
    R_i = inv_tri(R)
    Q_t = transpose(Q)
    return R_i*Q_t
end

B = [2 -2 18;
     7 124 9;
     9 2 2.0]

#######################################TEST
QRinv(QRinv(B))


3×3 Array{Float64,2}:
 2.0   -2.0  18.0
 7.0  124.0   9.0
 9.0    2.0   2.0